In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import psycopg2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
# This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. This study reviewed the literature and used the following 23 variables as explanatory variables:
# X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit.
# X2: Gender (1 = male; 2 = female).
# X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).
# X4: Marital status (1 = married; 2 = single; 3 = others).
# X5: Age (year).
# X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: X6 = the repayment status in September, 2005; X7 = the repayment status in August, 2005; . . .;X11 = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.
# X12-X17: Amount of bill statement (NT dollar). X12 = amount of bill statement in September, 2005; X13 = amount of bill statement in August, 2005; . . .; X17 = amount of bill statement in April, 2005. 
# X18-X23: Amount of previous payment (NT dollar). X18 = amount paid in September, 2005; X19 = amount paid in August, 2005; . . .;X23 = amount paid in April, 2005.

In [5]:
# Load environment variables
load_dotenv()

# PostgreSQL connection parameters
host = os.getenv("host")   
port = os.getenv("port")
dbname = os.getenv("dbname")      
user = os.getenv("user")
password = os.getenv("password")

# Establish database connection
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    cur = conn.cursor()

    # Load dataset from PostgreSQL
    query = "SELECT * FROM project_data"
    cur.execute(query)
    data = cur.fetchall()

    
    data_frame = pd.DataFrame(data, columns=[desc[0] for desc in cur.description])
    
   
except Exception as e:
    print(f"Error: {e}")

finally:
    # Close database connections
    if 'cur' in locals():
        cur.close()
    if 'conn' in locals():
        conn.close()

In [3]:
data_frame.shape

(30000, 25)

In [11]:
from dotenv import load_dotenv
import psycopg2
import os

# Load environment variables
load_dotenv()

# PostgreSQL connection parameters
host = os.getenv("host")   
port = os.getenv("port")
dbname = os.getenv("dbname")      
user = os.getenv("user")
password = os.getenv("password")

# Establish database connection
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    cur = conn.cursor()

    # Check data types of all columns
    cur.execute("""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = 'project_data';
    """)

    for column, dtype in cur.fetchall():
        print(f"{column}: {dtype}")

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'conn' in locals():
        conn.close()


ID: text
LIMIT_BAL: text
SEX: text
EDUCATION: text
MARRIAGE: text
AGE: text
PAY_0: text
PAY_2: text
PAY_3: text
PAY_4: text
PAY_5: text
PAY_6: text
BILL_AMT1: text
BILL_AMT2: text
BILL_AMT3: text
BILL_AMT4: text
BILL_AMT5: text
BILL_AMT6: text
PAY_AMT1: text
PAY_AMT2: text
PAY_AMT3: text
PAY_AMT4: text
PAY_AMT5: text
PAY_AMT6: text
default payment next month: text


In [ ]:
from dotenv import load_dotenv
import psycopg2
import os

# Load environment variables
load_dotenv()

# PostgreSQL connection parameters
host = os.getenv("host")
port = os.getenv("port")
dbname = "datafree"
user = os.getenv("user")
password = os.getenv("password")

try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    cur = conn.cursor()

    # Change "SEX" text values '1'/'2' → 'male'/'female'
    cur.execute("""
        UPDATE project_data
        SET "SEX" = CASE
            WHEN "SEX" = '1' THEN 'male'
            WHEN "SEX" = '2' THEN 'female'
            ELSE 'unknown'
        END;
    """)

    conn.commit()
    print("✅ Column \"SEX\" converted successfully!")

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'conn' in locals():
        conn.close()


✅ Column "SEX" converted successfully!


In [1]:
from dotenv import load_dotenv
import psycopg2
import os
import random

# Load environment variables
load_dotenv()

host = os.getenv("host")
port = os.getenv("port")
dbname = "datafree"
user = os.getenv("user")
password = os.getenv("password")

try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    cur = conn.cursor()

    # 1. Get all IDs
    cur.execute('SELECT "ID" FROM project_data;')
    ids = [row[0] for row in cur.fetchall()]

    # 2. Randomly pick 20 IDs
    random_ids = random.sample(ids, 30)

    # 3. Update rows to NULL for all columns except ID
    # First, get column names excluding ID
    cur.execute("""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_name = 'project_data'
          AND column_name <> 'ID';
    """)
    columns = [row[0] for row in cur.fetchall()]

    # Prepare SET part
    set_clause = ', '.join([f'"{col}" = NULL' for col in columns])

    # Update each randomly selected row
    for rid in random_ids:
        cur.execute(f'UPDATE project_data SET {set_clause} WHERE "ID" = %s;', (rid,))

    conn.commit()
    print("✅ 30 random rows set to NULL successfully.")

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'conn' in locals():
        conn.close()


✅ 30 random rows set to NULL successfully.
